In [579]:
import sys
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import torch
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, roc_auc_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',100)

# ====================================================
# Configurations
# ====================================================
class CFG:
    VER = 2.3
    AUTHOR = 'naokisusami'
    COMPETITION = 'FDUA2'
    DATA_PATH = Path('/data')
    OOF_DATA_PATH = Path('/oof')
    MODEL_DATA_PATH = Path('/models')
    SUB_DATA_PATH = Path('/submission')
    METHOD_LIST = ['lightgbm', 'xgboost', 'catboost']
    seed = 42
    n_folds = 7
    target_col = 'MIS_Status'
    metric = 'f1_score'
    metric_maximize_flag = True
    num_boost_round = 500
    early_stopping_round = 200
    verbose = 25
    classification_lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'seed': seed,
    }
    classification_xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': 0.05,
        'random_state': seed,
    }
    classification_cat_params = {
        'depth':7,
        'learning_rate': 0.03,
        'iterations': 500,
        'l2_leaf_reg': 1,
        'random_seed': seed,
    }
    model_weight_dict = {'lightgbm': 0.50, 'xgboost': 0.10, 'catboost': 0.40}
    '''
    classification_cat_params = {
        'learning_rate': 0.05,
        'iterations': num_boost_round,
        'random_seed': seed,
    }
    '''
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG.seed)


# ====================================================
# Metric
# ====================================================
# f1_score

# ====================================================
# LightGBM Metric
# ====================================================
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro'), CFG.metric_maximize_flag

# ====================================================
# XGBoost Metric
# ====================================================
def xgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'f1score', f1_score(y_true, np.where(y_pred >= 0.5, 1, 0), average='macro')


In [580]:
#データの読み込み
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)
categorical_features = ['RevLineCr', 'LowDoc', 'UrbanRural', 'State', 'Sector']

In [581]:
#前処理メソッドの定義
def Preprocessing(input_df: pd.DataFrame()) -> pd.DataFrame():
    #欠損値に対する処理
    def deal_missing(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['RevLineCr', 'LowDoc', 'BankState']:
            df[col] = input_df[col].fillna('UNK')
        for col in ['DisbursementDate','ApprovalDate']:
            df[col] = input_df[col].fillna('50-NaN-50')
        
        return df
    #金額に対する前処理
    def clean_money(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        for col in ['DisbursementGross', 'GrAppv', 'SBA_Appv']:
            df[col] = input_df[col].str[1:].str.replace(',', '').str.replace(' ', '').astype(float)
        return df
    df = deal_missing(input_df)
    df = clean_money(df)
    df['NewExist'] = np.where(input_df['NewExist'] == 1, 1, 0)
    #特徴量作成
    def make_features(input_df: pd.DataFrame()) -> pd.DataFrame():
        df = input_df.copy()
        #日付関係の特徴量作成
        df[['DisbursementDay','DisbursementMonth','DisbursementYear']] = df['DisbursementDate'].str.split('-',expand=True)
        df[['ApprovalDay','ApprovalMonth','ApprovalYear']] = df['ApprovalDate'].str.split('-',expand=True)
        df['DisbursementDay'] = df['DisbursementDay'].astype(int)
        df['DisbursementYear'] = df['DisbursementYear'].astype(int)
        df['ApprovalDay'] = df['ApprovalDay'].astype(int)
        df['ApprovalYear'] = df['ApprovalYear'].astype(int)
        Month_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'NaN':50}
        df['DisbursementMonth'] = df['DisbursementMonth'].map(Month_dict)
        df['ApprovalMonth'] = df['ApprovalMonth'].map(Month_dict)
        df['DisbursementDate'] = df['DisbursementYear'].astype(str)+df['DisbursementMonth'].astype(str)+df['DisbursementDay'].astype(str)
        df['DisbursementYear'] = df['DisbursementYear'].apply(lambda x:x - 100 if x >50 else x)
        df['ApprovalYear'] = df['ApprovalYear'].apply(lambda x:x - 100 if x >50 else x)
        df['CompanyLong'] = df['DisbursementYear'] - df['ApprovalYear']

        #Bankraptcydataの74~80は生成したもので実際の数値ではない。(失業率から換算して生成)
        Bankraptcydata={-26:32700,-25:52200,-24:46200,-23:42300,-22:36300,-21:34200,-20:46200,-19:44000,-18:48500,-17:69800,-16:62500,
                      -15:64500,-14:72000,-13:81500,-12:83000,-11:64500,-10:65000,-9:67000,-8:71000,-7:67000,-6:58000,-5:51000,
                        -4:52500,-3:54000,-2:51000,-1:41000,0:37500,1:35992,2:39845,3:37548,4:36785,5:31952,6:35292,7:21960,8:30741,
                        9:49091,10:61148,11:54212,12:46393,13:37552,14:31671,15:26130,16:24797,17:23591,18:23106,19:22157}

        #年ごとのデータを、1-5年後の平均に変換
        datalist = [Bankraptcydata]
        for k in datalist:
            for i in range(len(k)-5):
                k[-27+i] = 0
                for j in range(5):
                    k[-27+i] += k[-26+i+j]
                k[-27+i] = k[-27+i]/5
            k[50] = k[-26]*2
        
        df['Bankraptcy_By_Year'] = df['DisbursementYear'].map(Bankraptcydata)

        #組み合わせ特徴量
        df['State_Sector'] = df['State'].astype(str) + '_' + df['Sector'].astype(str)
         # 地理的特徴の組み合わせ
        df['City_State'] = df['City'] + '_' + df['State']
        # 時間的特徴の組み合わせ
        df['ApprovalFY_Term'] = df['ApprovalFY'].astype(str) + '_' + df['Term'].astype(str)
        
        df['FranchiseCode_ApprovalDate'] = df['FranchiseCode'].astype(str) + '_' + df['ApprovalDate'].astype(str)
        
        df['Term_NoEmp'] = df['Term'].astype(str) + '_' + df['NoEmp'].astype(str)
        
        df['City_BankState'] = df['City'].astype(str) + '_' + df['BankState'].astype(str)
        
         
        


        
        return df
    df = make_features(df)
    return df

In [582]:
#前処理の実行
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

In [583]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42307 entries, 0 to 42306
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Term                        42307 non-null  int64  
 1   NoEmp                       42307 non-null  int64  
 2   NewExist                    42307 non-null  int32  
 3   CreateJob                   42307 non-null  int64  
 4   RetainedJob                 42307 non-null  int64  
 5   FranchiseCode               42307 non-null  int64  
 6   RevLineCr                   42307 non-null  object 
 7   LowDoc                      42307 non-null  object 
 8   DisbursementDate            42307 non-null  object 
 9   MIS_Status                  42307 non-null  int64  
 10  Sector                      42307 non-null  int64  
 11  ApprovalDate                42307 non-null  object 
 12  ApprovalFY                  42307 non-null  int64  
 13  City                        423

（以下はPreprocessingに本来組み込むべきだが，コードが煩雑になるので，いったん切り出している．）

In [584]:
'''
#カウントエンコーディング
for col in categorical_features:
    count_dict = dict(train_df[col].value_counts())
    train_df[f'{col}_count_encoding'] = train_df[col].map(count_dict).astype(int)
    test_df[f'{col}_count_encoding'] = test_df[col].map(count_dict).fillna(1).astype(int)
'''

#ラベルエンコーディング
for col in categorical_features :
    le = LabelEncoder()
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    
categorical_features_unlabelable = ['ApprovalFY_Term','City_State','City','ApprovalDate','BankState','DisbursementDate','State_Sector',
                                   'FranchiseCode_ApprovalDate','Term_NoEmp','City_BankState']
'''
for col in categorical_features_unlabelable:
    le = LabelEncoder()   
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else len(le.classes_))
'''
for col in categorical_features_unlabelable:
    encoder = LabelEncoder()
    combined = pd.concat([train_df[col], test_df[col]], axis=0)
    encoder.fit(combined)
    train_df[col] = encoder.transform(train_df[col])
    test_df[col] = encoder.transform(test_df[col])

In [585]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42307 entries, 0 to 42306
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Term                        42307 non-null  int64  
 1   NoEmp                       42307 non-null  int64  
 2   NewExist                    42307 non-null  int32  
 3   CreateJob                   42307 non-null  int64  
 4   RetainedJob                 42307 non-null  int64  
 5   FranchiseCode               42307 non-null  int64  
 6   RevLineCr                   42307 non-null  int32  
 7   LowDoc                      42307 non-null  int32  
 8   DisbursementDate            42307 non-null  int32  
 9   MIS_Status                  42307 non-null  int64  
 10  Sector                      42307 non-null  int64  
 11  ApprovalDate                42307 non-null  int32  
 12  ApprovalFY                  42307 non-null  int64  
 13  City                        423

In [586]:
#OneHotEncoding
train_df2 = train_df.drop(['MIS_Status'],axis=1)
OneHotList = ['RevLineCr', 'LowDoc']
ohe = ce.OneHotEncoder(cols=OneHotList,use_cat_names=True)
train_df2 = ohe.fit_transform(train_df2)
test_df = ohe.transform(test_df)
train_df = pd.concat([train_df2,train_df['MIS_Status']],axis=1)

#featuresの作成
categorical_features = ['State', 'Sector','RevLineCr_0.0','RevLineCr_1.0','RevLineCr_2.0','RevLineCr_3.0','RevLineCr_4.0',
                       'LowDoc_0.0','LowDoc_1.0','LowDoc_2.0','LowDoc_3.0','LowDoc_4.0','LowDoc_5.0','LowDoc_6.0','FranchiseCode',
                       'ApprovalFY_Term','City_State','City','ApprovalDate','BankState','State_Sector','UrbanRural',
                        'FranchiseCode_ApprovalDate','Term_NoEmp','City_BankState']


RemoveList=['MIS_Status']
features = train_df.columns.tolist()
for i in RemoveList:
    print(i)
    features.remove(i)

MIS_Status


In [587]:
print(train_df)
print(features)

       Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCode  \
0       163     21         1          0            0              1   
1        84      6         1          4            0              0   
2       242     45         1          4           90              0   
3       237      4         1          0            0              0   
4       184      0         1          0            0              0   
...     ...    ...       ...        ...          ...            ...   
42302   283     14         1          0            0              1   
42303    53      2         1          0            0              0   
42304    59      6         0          0            0              1   
42305   295     18         1          0            8              0   
42306    84      4         1          0            8              0   

       RevLineCr_1.0  RevLineCr_0.0  RevLineCr_4.0  RevLineCr_3.0  \
0                  1              0              0              0   
1        


params = {'depth':[7],
         'learning_rate':[0.03],
         'l2_leaf_reg':[1],
         'iterations':[500],
         }

ctb = CatBoostClassifier(eval_metric='F1',logging_level='Silent')
ctb_grid_search = GridSearchCV(ctb,params,scoring='f1_macro',cv=3,verbose=2,error_score='raise')
ctb_grid_search.fit(train_df[features],train_df['MIS_Status'],cat_features=categorical_features)

print(ctb_grid_search.best_params_)
print(ctb_grid_search.best_index_)
print(ctb_grid_search.best_score_)


In [588]:
#lightgbmでの学習メソッドの定義
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
    lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
    model = lgb.train(
                params = CFG.classification_lgb_params,
                train_set = lgb_train,
                num_boost_round = CFG.num_boost_round,
                valid_sets = [lgb_train, lgb_valid],
                feval = lgb_metric,
                callbacks=[lgb.early_stopping(stopping_rounds=CFG.early_stopping_round,
                                              verbose=CFG.verbose)]
            )
    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

#xgboostでの学習メソッドの定義
def xgboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    xgb_train = xgb.DMatrix(data=x_train, label=y_train)
    xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid)
    model = xgb.train(
                CFG.classification_xgb_params,
                dtrain = xgb_train,
                num_boost_round = CFG.num_boost_round,
                evals = [(xgb_train, 'train'), (xgb_valid, 'eval')],
                early_stopping_rounds = CFG.early_stopping_round,
                verbose_eval = CFG.verbose,
                feval = xgb_metric,
                maximize = CFG.metric_maximize_flag,
            )
    # Predict validation
    valid_pred = model.predict(xgb.DMatrix(x_valid))
    return model, valid_pred

#catboostでの学習メソッドの定義
def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, features: list, categorical_features: list):
    cat_train = Pool(data=x_train, label=y_train, cat_features=categorical_features)
    cat_valid = Pool(data=x_valid, label=y_valid, cat_features=categorical_features)
    model = CatBoostClassifier(**CFG.classification_cat_params)
    model.fit(cat_train,
              eval_set = [cat_valid],
              early_stopping_rounds = CFG.early_stopping_round,
              verbose = CFG.verbose,
              use_best_model = True)
    # Predict validation
    valid_pred = model.predict_proba(x_valid)[:, 1]
    return model, valid_pred
#任意のモデルでのクロスバリデーション学習メソッドの定義
def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df)):
        print('-'*50)
        print(f'{method} training fold {fold+1}')

        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'xgboost':
            model, valid_pred = xgboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, features, categorical_features)

        # Save best model
        pickle.dump(model, open(f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'wb'))
        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = f1_score(train_df[CFG.target_col], oof_predictions >= 0.5, average='macro')
    print(f'{method} our out of folds CV f1score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    oof_df.to_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv', index = False)
#学習メソッドの定義
def Learning(input_df: pd.DataFrame, features: list, categorical_features: list):
    for method in CFG.METHOD_LIST:
        gradient_boosting_model_cv_training(method, input_df, features, categorical_features)

In [589]:
Learning(train_df, features, categorical_features)

--------------------------------------------------
lightgbm training fold 1
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 32336, number of negative: 3927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16828
[LightGBM] [Info] Number of data points in the train set: 36263, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.891708 -> initscore=2.108305
[LightGBM] [Info] Start training from score 2.108305

[0]	train-logloss:0.65988	train-f1score:0.09771	eval-logloss:0.65993	eval-f1score:0.09208
[25]	train-logloss:0.33438	train-f1score:0.67514	eval-logloss:0.33715	eval-f1score:0.64077
[50]	train-logloss:0.27716	train-f1score:0.67046	eval-logloss:0.28633	eval-f1score:0.63618
[75]	train-logloss:0.25892	train-f1score:0.67992	eval-logloss:0.27780	eval-f1score:0.64212
[100]	train-logloss:0.24879	train-f1score:0.68936	eval-logloss:0.27578	eval-f1score:0.64798
[125]	train-logloss:0.24142	train-f1score:0.69647	eval-logloss:0.27523	eval-f1score:0.65220
[150]	train-logloss:0.23552	train-f1score:0.70274	eval-logloss:0.27534	eval-f1score:0.65314
[175]	train-logloss:0.23193	train-f1score:0.70787	eval-logloss:0.27546	eval-f1score:0.65361
[200]	train-logloss:0.22822	train-f1score:0.71433	eval-logloss:0.27547	eval-f1score:0.65358
[225]	train-logloss:0.22422	train-f1score:0.71871	eval-logloss:0.27570	eval-f1score:0.65500
[250]	train-logloss:0.22064	train-f1score:0.72454	eval-logloss:0.27582	eval-f1score:0

--------------------------------------------------
xgboost training fold 7
[0]	train-logloss:0.65984	train-f1score:0.09712	eval-logloss:0.66014	eval-f1score:0.09563
[25]	train-logloss:0.33370	train-f1score:0.67150	eval-logloss:0.34097	eval-f1score:0.65989
[50]	train-logloss:0.27659	train-f1score:0.67002	eval-logloss:0.29155	eval-f1score:0.65435
[75]	train-logloss:0.25828	train-f1score:0.68002	eval-logloss:0.28242	eval-f1score:0.65740
[100]	train-logloss:0.24766	train-f1score:0.69034	eval-logloss:0.28015	eval-f1score:0.65883
[125]	train-logloss:0.24013	train-f1score:0.69810	eval-logloss:0.27998	eval-f1score:0.65858
[150]	train-logloss:0.23335	train-f1score:0.70421	eval-logloss:0.28018	eval-f1score:0.66158
[175]	train-logloss:0.22853	train-f1score:0.71010	eval-logloss:0.28010	eval-f1score:0.66310
[200]	train-logloss:0.22479	train-f1score:0.71494	eval-logloss:0.28022	eval-f1score:0.66173
[225]	train-logloss:0.22168	train-f1score:0.71843	eval-logloss:0.28034	eval-f1score:0.66286
[250]	trai

--------------------------------------------------
catboost training fold 4
0:	learn: 0.6641633	test: 0.6637097	best: 0.6637097 (0)	total: 590ms	remaining: 4m 54s
25:	learn: 0.3548859	test: 0.3490790	best: 0.3490790 (25)	total: 15s	remaining: 4m 32s
50:	learn: 0.2981576	test: 0.2916996	best: 0.2916996 (50)	total: 29.6s	remaining: 4m 20s
75:	learn: 0.2822039	test: 0.2766171	best: 0.2766171 (75)	total: 44.8s	remaining: 4m 10s
100:	learn: 0.2760622	test: 0.2718656	best: 0.2718656 (100)	total: 1m	remaining: 4m
125:	learn: 0.2727045	test: 0.2698615	best: 0.2698615 (125)	total: 1m 17s	remaining: 3m 49s
150:	learn: 0.2702898	test: 0.2688438	best: 0.2688438 (150)	total: 1m 33s	remaining: 3m 35s
175:	learn: 0.2681978	test: 0.2682362	best: 0.2682255 (174)	total: 1m 49s	remaining: 3m 21s
200:	learn: 0.2662018	test: 0.2678227	best: 0.2678227 (200)	total: 2m 5s	remaining: 3m 6s
225:	learn: 0.2644773	test: 0.2674810	best: 0.2674810 (225)	total: 2m 22s	remaining: 2m 52s
250:	learn: 0.2629519	test: 0.

catboost our out of folds CV f1score is 0.6462508994948974


In [590]:
def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(x_test)
        test_pred += pred
    return test_pred / CFG.n_folds
def xgboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'xgboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict(xgb.DMatrix(x_test))
        test_pred += pred
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.VER}.pkl', 'rb'))
        # Predict
        pred = model.predict_proba(x_test)[:, 1]
        test_pred += pred
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list, categorical_features: list):
    x_test = test_df[features]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test)
    if method == 'xgboost':
        test_pred = xgboost_inference(x_test)
    if method == 'catboost':
        test_pred = catboost_inference(x_test)
    return test_pred

def Predicting(input_df: pd.DataFrame, features: list, categorical_features: list):
    output_df = input_df.copy()
    output_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        output_df[f'{method}_pred_prob'] = gradient_boosting_model_inference(method, input_df, features, categorical_features)
        output_df['pred_prob'] += CFG.model_weight_dict[method] * output_df[f'{method}_pred_prob']
    return output_df

In [591]:
test_df = Predicting(test_df, features, categorical_features)

In [592]:

#後処理の定義
def Postprocessing(train_df: pd.DataFrame(), test_df: pd.DataFrame()) -> (pd.DataFrame(), pd.DataFrame()):
    train_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv')
        train_df['pred_prob'] += CFG.model_weight_dict[method] * oof_df[f'{method}_prediction']
    best_score = 0
    best_v = 0
    for v in tqdm(np.arange(1000) / 1000):
        score = f1_score(oof_df[CFG.target_col], train_df[f'pred_prob'] >= v, average='macro')
        if score > best_score:
            best_score = score
            best_v = v
    print(best_score, best_v)
    test_df['target'] = np.where(test_df['pred_prob'] >= best_v, 1, 0)
    return train_df, test_df

In [593]:
'''
#後処理の定義、調和平均版 
def Postprocessing(train_df: pd.DataFrame(), test_df: pd.DataFrame()) -> (pd.DataFrame(), pd.DataFrame()): 
    train_df['pred_prob'] = 0 
    weight_sum = 0 
    for method in CFG.METHOD_LIST: 
        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv') 
        train_df['pred_prob'] += CFG.model_weight_dict[method] / oof_df[f'{method}_prediction'] 
        weight_sum += CFG.model_weight_dict[method] 
    train_df['pred_prob'] = weight_sum / train_df['pred_prob'] 
    best_score = 0 
    best_v = 0 
    for v in tqdm(np.arange(1000) / 1000):
        score = f1_score(oof_df[CFG.target_col], train_df[f'pred_prob'] >= v, average='macro') 
        if score > best_score: 
            best_score = score 
            best_v = v 
    print(best_score, best_v) 
    test_df['target'] = np.where(test_df['pred_prob'] >= best_v, 1, 0)
    return train_df, test_df
'''

"\n#後処理の定義、調和平均版 \ndef Postprocessing(train_df: pd.DataFrame(), test_df: pd.DataFrame()) -> (pd.DataFrame(), pd.DataFrame()): \n    train_df['pred_prob'] = 0 \n    weight_sum = 0 \n    for method in CFG.METHOD_LIST: \n        oof_df = pd.read_csv(f'oof_{method}_seed{CFG.seed}_ver{CFG.VER}.csv') \n        train_df['pred_prob'] += CFG.model_weight_dict[method] / oof_df[f'{method}_prediction'] \n        weight_sum += CFG.model_weight_dict[method] \n    train_df['pred_prob'] = weight_sum / train_df['pred_prob'] \n    best_score = 0 \n    best_v = 0 \n    for v in tqdm(np.arange(1000) / 1000):\n        score = f1_score(oof_df[CFG.target_col], train_df[f'pred_prob'] >= v, average='macro') \n        if score > best_score: \n            best_score = score \n            best_v = v \n    print(best_score, best_v) \n    test_df['target'] = np.where(test_df['pred_prob'] >= best_v, 1, 0)\n    return train_df, test_df\n"

In [594]:
#後処理
train_df, test_df = Postprocessing(train_df, test_df)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.6878680332744743 0.758


In [595]:
test_df[['target']].to_csv(f'seed{CFG.seed}_ver{CFG.VER}_{CFG.AUTHOR}_submission.csv', header=False)

特徴量の重要度を確認する方法

In [596]:
model = pickle.load(open(f'lightgbm_fold1_seed42_ver{CFG.VER}.pkl', 'rb'))
importance_df = pd.DataFrame(model.feature_importance(), index=features, columns=['importance'])
importance_df['importance'] = importance_df['importance'] / np.sum(importance_df['importance'])
importance_df.sort_values('importance', ascending=False)

,importance
City_BankState,0.150000
Term_NoEmp,0.125397
ApprovalFY_Term,0.125397
State_Sector,0.124603
ApprovalDate,0.112698
FranchiseCode_ApprovalDate,0.092857
NoEmp,0.034921
UrbanRural,0.027778
LowDoc_4.0,0.023016
Term,0.023016
